In [ ]:
## Notebook settings
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
sys.path.append("../")
from audio import *
import random
from fastai.callbacks import *
from IPython.core.debugger import set_trace
np.random.seed(2)
p = Config.data_path()/'freesound-audio-tagging-2019';
print(p)

In [ ]:
# !kaggle competitions download -c freesound-audio-tagging-2019 -p {p}

In [ ]:
train_path = p/'train_noisy'
valid_path = p/'train_curated'
test_path = p/'test'

In [ ]:
config = AudioTransformConfig()
config.remove_silence = True
config.silence_threshold = 5
config.silence_padding = 1000
config.sg_cfg.f_max = 44100//2
# config.segment_size = 2000
# config.max_segments = 2
# config.window_size = 2000

# For 224x224 nmels = 224, hop 395
# For 128x127 hop 694
config.sg_cfg.n_mels = 128
config.sg_cfg.n_fft = 224*12
config.sg_cfg.hop = 694
config.to_db_scale = True
config.top_db = 76
config.max_to_pad = 2000
items = ItemList.from_csv(valid_path, '../train_curated.csv').split_none().label_from_df(cols='labels', label_delim=',')
carr = []
arr = []

# Get one from each class
for i in range(len(items.x)):
    if not str(items.y[i]) in carr:
        carr.append(str(items.y[i]))
        arr.append(items.x[i])

        
print(len(arr))
# DODGY: TAP (21) Church Bell(37)  Clapping (88)

def preview_tfms(idx):
    print("Class:", carr[idx])
    prevs = preview_transforms(arr[idx], valid_path, config)
    for i,y in prevs: i.show(); print(i.data.shape)
        
preview_tfms(88)

In [ ]:
def _match_cond(data, cond):
    return [i for i, x in enumerate(data) if cond(x)]

def nonmulti(items):
    indexs = _match_cond(items, lambda x : len(x[1].obj)<2)
    return data.train[indexs]


In [ ]:
data = AudioList.from_csv(valid_path, '../train_curated.csv', config=config).split_by_rand_pct(.2, seed=2).label_from_df(cols='labels', label_delim=',')
data.train = nonmulti(data.train)
# data.valid = nonmulti(data.valid)
tfms = get_spectro_transforms(roll=True, num_rows=5, num_cols=5, tmasks=2, max_shift_pct=.1)
db = data.transform(tfms).databunch(bs=64)
print(db)
db.show_batch()

In [ ]:
# from official code https://colab.research.google.com/drive/1AgPdhSp7ttY18O3fEoHOQKlt_3HJDLi8#scrollTo=cRCaCIb9oguU
def _one_sample_positive_class_precisions(scores, truth):
    """Calculate precisions for each true class for a single sample.

    Args:
      scores: np.array of (num_classes,) giving the individual classifier scores.
      truth: np.array of (num_classes,) bools indicating which classes are true.

    Returns:
      pos_class_indices: np.array of indices of the true classes for this sample.
      pos_class_precisions: np.array of precisions corresponding to each of those
        classes.
    """
    num_classes = scores.shape[0]
    pos_class_indices = np.flatnonzero(truth > 0)
    # Only calculate precisions if there are some true classes.
    if not len(pos_class_indices):
        return pos_class_indices, np.zeros(0)
    # Retrieval list of classes for this sample.
    retrieved_classes = np.argsort(scores)[::-1]
    # class_rankings[top_scoring_class_index] == 0 etc.
    class_rankings = np.zeros(num_classes, dtype=np.int)
    class_rankings[retrieved_classes] = range(num_classes)
    # Which of these is a true label?
    retrieved_class_true = np.zeros(num_classes, dtype=np.bool)
    retrieved_class_true[class_rankings[pos_class_indices]] = True
    # Num hits for every truncated retrieval list.
    retrieved_cumulative_hits = np.cumsum(retrieved_class_true)
    # Precision of retrieval list truncated at each hit, in order of pos_labels.
    precision_at_hits = (
            retrieved_cumulative_hits[class_rankings[pos_class_indices]] /
            (1 + class_rankings[pos_class_indices].astype(np.float)))
    return pos_class_indices, precision_at_hits


def calculate_per_class_lwlrap(truth, scores):
    """Calculate label-weighted label-ranking average precision.

    Arguments:
      truth: np.array of (num_samples, num_classes) giving boolean ground-truth
        of presence of that class in that sample.
      scores: np.array of (num_samples, num_classes) giving the classifier-under-
        test's real-valued score for each class for each sample.

    Returns:
      per_class_lwlrap: np.array of (num_classes,) giving the lwlrap for each
        class.
      weight_per_class: np.array of (num_classes,) giving the prior of each
        class within the truth labels.  Then the overall unbalanced lwlrap is
        simply np.sum(per_class_lwlrap * weight_per_class)
    """
    assert truth.shape == scores.shape
    num_samples, num_classes = scores.shape
    # Space to store a distinct precision value for each class on each sample.
    # Only the classes that are true for each sample will be filled in.
    precisions_for_samples_by_classes = np.zeros((num_samples, num_classes))
    for sample_num in range(num_samples):
        pos_class_indices, precision_at_hits = (
            _one_sample_positive_class_precisions(scores[sample_num, :],
                                                  truth[sample_num, :]))
        precisions_for_samples_by_classes[sample_num, pos_class_indices] = (
            precision_at_hits)
    labels_per_class = np.sum(truth > 0, axis=0)
    weight_per_class = labels_per_class / float(np.sum(labels_per_class))
    # Form average of each column, i.e. all the precisions assigned to labels in
    # a particular class.
    per_class_lwlrap = (np.sum(precisions_for_samples_by_classes, axis=0) /
                        np.maximum(1, labels_per_class))
    # overall_lwlrap = simple average of all the actual per-class, per-sample precisions
    #                = np.sum(precisions_for_samples_by_classes) / np.sum(precisions_for_samples_by_classes > 0)
    #           also = weighted mean of per-class lwlraps, weighted by class label prior across samples
    #                = np.sum(per_class_lwlrap * weight_per_class)
    return per_class_lwlrap, weight_per_class


# Wrapper for fast.ai library
def lwlrap(scores, truth, **kwargs):
    score, weight = calculate_per_class_lwlrap(to_np(truth), to_np(scores))
    return torch.Tensor([(score * weight).sum()])

In [ ]:
learn = audio_learner(db, models.resnet18, metrics=[lwlrap], pretrained=True, callback_fns=[CSVLogger, ShowGraph])

In [ ]:
learn.lr_find(); learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(20, slice(1e-3, 1e-2))
learn.save('stage-1')

In [ ]:
# learn.load('stage-1')
data = AudioList.from_csv(valid_path, '../train_curated.csv', config=config).split_by_rand_pct(.2, seed=2).label_from_df(cols='labels', label_delim=',')

nmulti = nonmulti(data.train)

newitems = []
for _ in range(3000): 
    d1 = nmulti[random.randint(0,len(nmulti)-1)]
    d2 = nmulti[random.randint(0,len(nmulti)-1)]
    mark = config.cache_dir / ("c_"+md5(str(d1[0].path)+str(d2[0].path))+'.wav')
    d1[0].sig += d2[0].sig
    d1[1].obj += d2[1].obj
    d1[1].data += d2[1].data
    d1[1].raw += d2[1].raw
    d1[0].path = mark
    torchaudio.save(str(mark), d1[0].sig[None,:], d1[0].sr)
    print(d1[1].raw)
    data.train.y.items = np.append(data.train.y.items, l, axis=0)
    print(data.train.y.items)
#     data.train.x.items = np.append(data.train.x.items, mark)

tfms = get_spectro_transforms(roll=True, num_rows=10, num_cols=10, tmasks=2, max_shift_pct=.1)
db = data.transform(tfms).databunch(bs=64)
learn.data = db
db

In [ ]:
learn.lr_find(); learn.recorder.plot()

In [ ]:

learn.fit_one_cycle(10, slice(1e-4, 1e-3))

In [ ]:
learn.lr_find(); learn.recorder.plot()

In [ ]:
test = ItemList.from_csv(test_path, '../sample_submission.csv').split_none().label_from_df()

In [ ]:
for i,l in test.train:
    ai = AudioItem.open(test_path/i)
    ai.show()
    print(audio_predict(learn, ai))